In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, get_paper_data, mod_names, check_google, similar_names, search_names
from modules.data_manipulation_functions import remove_commas, check_files, concat_function


In [2]:

# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)
g_driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")

In [12]:
# List the files that are in the intermediate directory
# This is where we will be storing the individual output files
#These three lines give a list of the files that have already been scraped
#Papers gives a list of the papers by each author
#Skip gives a list of the skipped names
current_data = os.listdir(intr_path)

# Load the datasets from the working directory
# The datasets in the working directory have already 
# been cleaned.
control = pd.read_excel(work_path / "control.xlsx")
lateral = pd.read_excel(work_path / "lateral_test.xlsx")

# Load the name modification dataset
# This is a dataframe of names that we want to manually change 
# This can be used if we found errors in the data (for example, names were not scraped)
# This dataset is edited by hand, so it is stored in the input directory
name_mod = pd.read_excel(input_path / 'name_mod.xlsx')

# Data type: This is the type of data that we are using.
# The control group members only have one university url because they
# did not move. The lateral group members have two urls.
data_type = "lateral"

#This step sets the data 
data = lateral


In [13]:
# Create a list to store the alternate name data
alt_names_data = []
# Create the alt-names dataset
alt_names_columns = ["ID", "mid_first_name", "last_name", "fm_names", "err_fm_names", "diff_last_name"]
# Add the names back into the dataframe of alternate names
df_alt_names = pd.DataFrame(columns = alt_names_columns)

# Check to see if the file for the alternate names data already exists
alt_names_file = intr_path / "alt_names.xlsx"
print(alt_names_file)
print(alt_names_file.exists())
if alt_names_file.exists():
    append = 1
    df_alt_names_final = pd.read_excel(alt_names_file)
    # alt_name_full = pd.merge(df_alt_names, lateral, how = "right", left_on = "ID", right_on = "ID")
    data = pd.merge(data, df_alt_names_final["ID"], how = "outer", left_on = "ID", right_on = "ID", indicator=True)
    data = data[data['_merge'] == 'left_only']
    data = data.drop(["_merge"], axis = 1)
else:
    append = 0
data.reset_index(inplace = True)

c:\Users\marti\OneDrive\Documents\GitHub\hein_scraping\data\b_intermediate\alt_names.xlsx
True


In [14]:
#This loop goes through each name
for i in range(len(data)):
    #This section gets the professor's information from the dataframe 
    # Get variable values from the dataframe
    prof_id = data['ID'][i]
    mid_first_name = data['FirstName'][i]
    last_name = data['LastName'][i]
    full_name = mid_first_name + ' ' +  last_name
    #This line gets the school URLs from the dataframe
    if data_type == "lateral":
        school_url = [data['Short URL Origin'][i], data['Short URL Destination'][i]]
        school = data['Origin School'][i]
        new_school = data['Destination School'][i]
    elif data_type == "control":
        school_url = [data['Short URL Origin'][i]]
        school = data['Origin School'][i]

    # check_files checks if a file for the papers from the professor has already been created.
    #If a file has already been created for the professor, the loop moves onto the next name.
    if check_files(mid_first_name, last_name, current_data):
        print('File for ' + full_name + ' has already been created.')
        continue

    # Print the name that we are considering
    print(full_name)
    
    # Search by author to find potential alternative first and middle names:
    print(school_url)
    fm_names, err_fm_names = search_names(mid_first_name, last_name, school_url, driver, g_driver)


    # Create a list of values to append to the dataframe
    # We convert fm_names and err_fm_names to lists of strings during this step
    values_alt_names = [prof_id, mid_first_name, last_name, fm_names, err_fm_names]
    dict_values_alt_names = dict(zip(alt_names_columns, values_alt_names))
    alt_names_data.append(dict_values_alt_names)



 'duke.edu']
Updated
URL: vanderbilt.edu
Element_text: https://www.bing.com/?FORM=Z9FD1
Element_text: javascript:void(0);
Element_text: javascript:void(0);
Element_text: javascript:void(0)
Element_text: https://www.bing.com/rewards/dashboard
Element_text: javascript:void(0);
Element_text: https://www.bing.com/images/search?q=Nita+A.+Farahany+&FORM=HDRSC2
Element_text: https://www.bing.com/videos/search?q=Nita+A.+Farahany+&FORM=HDRSC3
Element_text: https://www.bing.com/maps?q=Nita+A.+Farahany+&FORM=HDRSC4
Element_text: https://www.bing.com/news/search?q=Nita+A.+Farahany+&FORM=HDRSC6
Element_text: https://www.bing.com/shop?q=Nita+A.+Farahany+&FORM=SHOPTB
Element_text: javascript:
Element_text: https://law.duke.edu/fac/farahany/
Element_text: https://www.bing.com/images/search?q=nita+a.+farahany&qpvt=Nita+A.+Farahany&FORM=IGRE
Element_text: https://www.bing.com/images/search?q=nita+a.+farahany&id=CB433B760076B56DD41E7792D3C4465B8CD8A2DD&FORM=IQFRBA
Element_text: https://www.bing.com/image

In [16]:
# Append all of the data to the ouput dataset and output to Excel
df_alt_names = df_alt_names.append(alt_names_data)
# MERGE: Merge on the other variables from the lateral data
alt_name_full = pd.merge(df_alt_names, lateral, how = "left", left_on = "ID", right_on = "ID")
# MERGE: Merge on the name mod dataset
alt_name_name_mod_merge = pd.merge(alt_name_full, name_mod, how = "left", left_on = ["mid_first_name", "last_name"], right_on = ["mid_first_name", "last_name"], suffixes=('_orig', '_mod'))
alt_name_name_mod_merge
# Concatenate the values from the alternate name and name mod dataframes
alt_name_name_mod_merge["fm_names"] = alt_name_name_mod_merge.apply(lambda x: concat_function(x["fm_names_orig"], x["fm_names_mod"]), axis = 1) 
alt_name_name_mod_merge["err_fm_names"] = alt_name_name_mod_merge.apply(lambda x: concat_function(x["err_fm_names_orig"], x["err_fm_names_mod"]), axis = 1)
alt_name_name_mod_merge["diff_last_name"] = alt_name_name_mod_merge.apply(lambda x: concat_function(x["diff_last_name_orig"], x["diff_last_name_mod"]), axis = 1) 
# DROP: Drop the duplicate columns from the alternate name and name mod dataframes
alt_name_name_mod_merge = alt_name_name_mod_merge.drop(["fm_names_orig", "err_fm_names_orig", "diff_last_name_orig", "fm_names_mod", "err_fm_names_mod", "diff_last_name_mod"], axis = 1)
# If there was already output data, we append the data to the existing data
if append == 1:
    alt_name_name_mod_merge = pd.concat([alt_name_name_mod_merge, df_alt_names_final], ignore_index = True)
# Drop unnamed columns
alt_name_name_mod_merge.drop(alt_name_name_mod_merge.columns[alt_name_name_mod_merge.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# Output to Excel
alt_name_name_mod_merge.to_excel(intr_path / "alt_names.xlsx")

,ID,mid_first_name,last_name,fm_names_orig,err_fm_names_orig,diff_last_name_orig,Unnamed: 0,Short URL Destination,Short URL Origin,FirstName,LastName,Lateral,LateralYear,Origin School,Destination School,fm_names_mod,err_fm_names_mod,diff_last_name_mod
0,71,Nita,Farahany,"Nita A., Nita",,NaN,6,duke.edu,vanderbilt.edu,Nita,Farahany,1,2012,Vanderbilt University Law School,Duke University School of Law,NaN,NaN,NaN
1,146,Jody,Kraus,"Jody S., Jody",,NaN,7,columbia.edu,upenn.edu,Jody,Kraus,1,2012,University of Pennsylvania Law School,Columbia Law School,NaN,NaN,NaN
2,22,Anu,Bradford,Anu,,NaN,8,columbia.edu,uchicago.edu,Anu,Bradford,1,2012,University of Chicago Law School,Columbia Law School,NaN,NaN,NaN
